In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load and prepare the dataset
file_path = 'goal_ticks.csv'
column_names = [
    "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "ball_appr_goal", "dist_to_ball",
    "kart_X", "kart_Z", "vel_X", "vel_Z", "speed", "steer", "accel", "brake", "skid", "goal"
]

df = pd.read_csv(file_path, header=None, names=column_names)

# Convert boolean column 'ball_appr_goal' to float
df['ball_appr_goal'] = df['ball_appr_goal'].astype(float)

# Extract features and labels
X = df.iloc[:, :-1].values  # All columns except the last one
y = df['goal'].values  # The 'goal' column as binary

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Keeping the target as a 2D tensor for BCELoss

# Define the neural network architecture
class ScoringModel(nn.Module):
    def __init__(self, num_inputs):
        super(ScoringModel, self).__init__()
        self.layer1 = nn.Linear(num_inputs, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 64)
        self.layer5 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.leaky_relu(self.layer2(x))
        x = self.leaky_relu(self.layer3(x))
        x = self.leaky_relu(self.layer4(x))
        x = self.leaky_relu(self.layer5(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Initialize the model
model = ScoringModel(X_tensor.shape[1])

# Set up the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Prepare data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(train_loader, model, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Validation function
def validate_model(val_loader, model, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f'Validation Loss: {total_loss / len(val_loader):.4f}')

# Train and validate the model
train_model(train_loader, model, criterion, optimizer, 20)
validate_model(val_loader, model, criterion)

# Save the trained model for deployment
model.eval()
example_input = torch.rand(1, X_train.shape[1])  # Generate a random example input
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("soccer_ai_model.pt")


Epoch 1/20, Loss: 0.0492
Epoch 2/20, Loss: 0.2189
Epoch 3/20, Loss: 0.0597
Epoch 4/20, Loss: 0.1952
Epoch 5/20, Loss: 0.0442
Epoch 6/20, Loss: 0.0441
Epoch 7/20, Loss: 0.0993
Epoch 8/20, Loss: 0.2570
Epoch 9/20, Loss: 0.0053
Epoch 10/20, Loss: 0.2841
Epoch 11/20, Loss: 0.0109
Epoch 12/20, Loss: 0.0146
Epoch 13/20, Loss: 0.1820
Epoch 14/20, Loss: 0.0301
Epoch 15/20, Loss: 0.0205
Epoch 16/20, Loss: 0.0395
Epoch 17/20, Loss: 0.0198
Epoch 18/20, Loss: 0.0428
Epoch 19/20, Loss: 0.3285
Epoch 20/20, Loss: 0.1779
Validation Loss: 0.1095
